In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
number_of_month = 3 # Set the index for number of month(August, September, October)
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
unit_cost = [90, 95, 90]  # Bike cost of per unit in each month  
unit_holding_cost = 5  # Storage/Holding cost incurred per unit  
demand = [40, 45, 35]   # Demand in each month 
unit_selling_price = [90, 95, 99]  # Bike selling price per unit in each month
initial_inventory = 10  # Inventory at the beginning of month August

In [4]:
# Set the Variable list: Units of bikes to be ordered  in each month
# Set the variable nx to integer number
nx = [] 
for i in range(number_of_month):
    nx.append(m.addVar(vtype=GRB.INTEGER, name='nx{}'.format(i + 1)))

# Caculate inventory at the end of each month
ni = []
for i in range(len(nx)):
    if i > 0:
        initial_inventory = ni[i-1]
    xx = nx[i] + initial_inventory - demand[i]
    ni.append(xx)

In [5]:
# Set the Maximize Obijective: Net Profit
m.setObjective(quicksum([unit_selling_price[i]*demand[i] for i in range(len(nx))]) - quicksum([unit_cost[i]*nx[i] for i in range(len(nx))]) - quicksum([unit_holding_cost*ni[i] for i in range(len(nx))]),  GRB.MAXIMIZE)

In [6]:
# Set Non Negative decision variable
c1 = []
for i in range(len(nx)):
    c1.append(m.addConstr(nx[i] >= 0))
    
# End of month inventory ni is non negative 
c2 = []
for i in range(len(nx)):
    c2.append(m.addConstr(ni[i] >= 0))

In [7]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 6 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x45a26774
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+02]
Presolve removed 6 rows and 3 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2150000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds
Optimal objective  1.215000000e+03


In [8]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
         nx1           75 
         nx3           35 


In [9]:
# Get the Optimal Objective Value
m.ObjVal

1215.0

In [10]:
# Get the optimal solution list
m.X

[75.0, 0.0, 35.0]

In [11]:
# Get the end of inventory for each month
nin = []
initial_inventory = 10
for i in range(3):
    if i > 0:
        initial_inventory = nin[i-1]
    xx = m.X[i] + initial_inventory - demand[i]
    nin.append(xx)
print(nin)

[45.0, 0.0, 0.0]
